<a href="https://colab.research.google.com/github/Gutobastos/imersao-iA-3-Alura/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
%pip install google-genai

import os

google_api_key = os.environ.get('GOOGLE_API_KEY')

if google_api_key:
    print("Chave da API do Google encontrada nas variáveis de ambiente do seu PC.\n")
else:
    print("Chave da API do Google não encontrada nas variáveis de ambiente.\n")

from google import genai
client = genai.Client()
modelo = "gemini-2.0-flash"

from google.genai import types
chat_config = types.GenerateContentConfig(
    system_instruction = "Voce é um assistente pessoal sempre responda de forma educada e sucinta.",
)

prompt = input("Esperando a pergunta: ")
while prompt != "fim":
  chat = client.chats.create(model = modelo, config = chat_config)
  resposta = chat.send_message(prompt)
  print("Resposta: ", resposta.text, "\n")
  prompt = input("Esperando a pergunta ou escreva 'fim' para terminar: ")

print ("Fim do programa!")


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Chave da API do Google encontrada nas variáveis de ambiente do seu PC.

Resposta:  Olá! Como posso ajudar você hoje?
 

Resposta:  Existem algumas opções para encontrar sofás de 2 lugares mais baratos:

*   **Lojas de departamento e grandes varejistas:** Muitas vezes oferecem promoções e descontos em móveis.
*   **Lojas de móveis com preços acessíveis:** Algumas lojas se especializam em móveis mais baratos, sem comprometer muito a qualidade.
*   **Brechós e lojas de segunda mão:** Você pode encontrar ótimas ofertas em sofás usados em bom estado.
*   **Mercados online e classificados:** Sites como Mercado Livre e OLX podem ter opções de sofás usados ou novos a preços mais baixos.
*   **Lojas de outlet de móveis:** Estas lojas vendem produtos de mostruário, ponta de estoque ou com pequenos defeitos a preços reduzidos.

Ao pesquisar, compare preço